In [1]:
# Simple thermostat API using FastAPI
# importing libraries
from fastapi import FastAPI, Depends
import uvicorn
import nest_asyncio
import random
import json

app = FastAPI()


In [2]:
# Defining the thermostat class
class Thermostat:
    def __init__(self,
                 name: str,
                 temperature: int,
                 humidity: int,
                 mode: str):
        self.name = name
        self.temperature = temperature
        self.humidity = humidity
        self.mode = mode

    def get_temperature(self):
        return self.temperature

    def get_humidity(self):
        return self.humidity

    def get_mode(self):
        return self.mode

    def get_name(self):
        return self.name

    def get_json(self):
        return self.__dict__

    def set_temperature(self, temperature: int):
        self.temperature = temperature

    def set_humidity(self, humidity: int):
        self.humidity = humidity

    def set_mode(self, mode: str):
        self.mode = mode

    def set_name(self, name: str):
        self.name = name


In [3]:
# Creating a thermostat object
thermostat = Thermostat(name="Thermostat",
                        temperature=20,
                        humidity=50,
                        mode="cool")

In [4]:
thermostat.get_json()

{'name': 'Thermostat', 'temperature': 20, 'humidity': 50, 'mode': 'cool'}

In [5]:
# Defining the API endpoints
@app.get("/")
async def root():
    return {"message": "Welcome to the thermostat API"}

@app.get("/thermostat")
async def get_thermostat():
    return thermostat.get_json()

@app.get("/thermostat/temperature")
async def get_temperature():
    return thermostat.get_temperature()

@app.get("/thermostat/humidity")
async def get_humidity():
    return thermostat.get_humidity()

@app.get("/thermostat/mode")
async def get_mode():
    return thermostat.get_mode()

@app.get("/thermostat/name")
async def get_name():
    return thermostat.get_name()

@app.put("/thermostat/temperature")
async def set_temperature(temperature: int):
    thermostat.set_temperature(temperature)
    return thermostat.get_temperature()

@app.put("/thermostat/humidity")
async def set_humidity(humidity: int):
    thermostat.set_humidity(humidity)
    return thermostat.get_humidity()

@app.put("/thermostat/mode")
async def set_mode(mode: str):
    thermostat.set_mode(mode)
    return thermostat.get_mode()

@app.put("/thermostat/name")
async def set_name(name: str):
    thermostat.set_name(name)
    return thermostat.get_name()


In [ ]:
# Running the API
if __name__ == "__main__":
    nest_asyncio.apply()
    #uvicorn.run(app, host="0.0.0.0", port=8001)
    uvicorn.run(app)

INFO:     Started server process [279]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     172.17.0.1:39198 - "GET / HTTP/1.1" 200 OK
INFO:     172.17.0.1:36030 - "GET /docks HTTP/1.1" 404 Not Found
INFO:     172.17.0.1:36030 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.17.0.1:36030 - "GET /openapi.json HTTP/1.1" 200 OK
